## Scraping module
Scraping of straits times news article using selenium that are returned from the search function (keyword - 'food supply')

### Imports

In [ ]:
#!pip install nltk
#import nltk
#nltk.download('punkt')

In [ ]:
#!pip install newspaper3k

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time

from newspaper import Article
import pandas as pd
import datetime

### Newspaper Article Summariser

In [2]:
def getSummary(url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    full_text = article.text
    location = full_text[0:full_text.find("-")]
    return article.summary, article.keywords, article.publish_date, location

### StraitsTimes Scraper

In [5]:
# change to ur own directory!
driver = webdriver.Chrome("C:/Users/User/Desktop/FYP/chromedriver.exe") 

def getNews():
    output = []
    
    # get news title in a page - need a for loop to get all the pages!
    all_news_per_page=WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME,"queryly_item_title")))
    
    # get news url (i.e. news source) & news summary & keywords using the summariser module
    for i in range(len(all_news_per_page)):
        news_details = []
        xpath = f'//*[@id="resultdata"]/div[{i+2}]/a'        
        url = all_news_per_page[i].find_element_by_xpath(xpath).get_attribute('href')
        try: 
            summary = getSummary(url)
            try:
                date = summary[2].strftime('%d/%m/%Y')[3:]
            except:
                date = "NA"
            news_details.append(date)
            news_details.append(summary[3])
            news_details.append(all_news_per_page[i].text)
            news_details.append(url)
            news_details.append(summary[0])
            news_details.append(", ".join(summary[1]))
            output.append(news_details)
        except: 
            continue
        
    return output

In [17]:
def getStraitsTimesNews(searchTerm):
    base_url = "https://www.straitstimes.com/search?searchkey=" + searchTerm.replace(" ", "%20")
    driver.get(base_url)

    # close ad (if any)
    try:
        iframe=driver.find_elements_by_tag_name("iframe")[3]
        driver.switch_to.frame(iframe)
        driver.find_elements_by_tag_name("iframe")
        ad = driver.switch_to.frame(0)
        driver.find_element_by_id('pclose-btn').click()
        driver.switch_to.default_content()
    except:
        driver.switch_to.default_content()
    
    # filter the search result by date by xpath accordingly
    # within 24 hours, within one week, within 30 days, within a year, within three years
#     filter_by_date = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.XPATH,"//*[contains(text(), 'within three years')]")))[0].click()

    # get news on page
    output = getNews()
    
    # check for next page
    nextpage = True
    while (nextpage):  
        try:
            nextpage = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,'#resultdata > a > h2')))[0]
            if (nextpage.text == 'Next Page'):
                nextpage.click()
#                 print('clicked next')
                output.extend(getNews())
            else: 
                nextpage = False
        except:
#             print("no next page!")
            nextpage = False
    
    
    # convert to df and export as csv
    df = pd.DataFrame(output, columns=['date', 'location','news title', 'news source(url)', 'content summary', 'keywords'])
    df['class_name'] = "Severe weather disruptions"
    df.to_csv(f"csv_data/{searchTerm}_straitstimes.csv")
    return df

getStraitsTimesNews("hailstorms")

,date,location,news title,news source(url),content summary,keywords,class_name
0,07/2019,GUADALAJARA (Mexico) • A freak hailstorm struc...,Mexican city hit by freak hailstorm,https://www.straitstimes.com/world/americas/me...,GUADALAJARA (Mexico) • A freak hailstorm struc...,"freak, reported, guadalajara, mexican, hit, ci...",Severe weather disruptions
1,12/2018,Sydney was hit yesterday by a violent hailstor...,Violent hailstorm cripples Sydney,https://www.straitstimes.com/world/violent-hai...,Sydney was hit yesterday by a violent hailstor...,"widespread, sydney, damage, car, hail, violent...",Severe weather disruptions
2,07/2019,TIGNES • Colombian Egan Bernal snatched the To...,Bernal takes yellow as hailstorm halts stage,https://www.straitstimes.com/sport/bernal-take...,TIGNES • Colombian Egan Bernal snatched the To...,"yellow, riders, bernal, timings, takes, hailst...",Severe weather disruptions
3,12/2018,SYDNEY,Widespread damage in Sydney after violent hail...,https://www.straitstimes.com/asia/australianz/...,SYDNEY - Sydney was hit by a violent hailstorm...,"sydney, damage, hailstones, car, violent, city...",Severe weather disruptions
4,02/2019,(REUTERS),Golf: Hailstorm causes final round delay at Pe...,https://www.straitstimes.com/sport/golf/golf-h...,(REUTERS) - The Pebble Beach Pro-Am appears he...,"causes, leaders, won, proam, pebble, golf, 15,...",Severe weather disruptions
5,05/2018,BORDEAUX • Violent hailstorms ravaged parts of...,Hailstorms ravage parts of France's Bordeaux v...,https://www.straitstimes.com/world/europe/hail...,BORDEAUX • Violent hailstorms ravaged parts of...,"vineyard, region, affected, ravage, hailstorms...",Severe weather disruptions
6,07/2019,"TIGNES, France (REUTERS)",Cycling: Bernal takes Tour de France yellow je...,https://www.straitstimes.com/sport/cycling-ber...,"TIGNES, France (REUTERS) - Egan Bernal looked ...","title, rider, set, lead, cycling, bernal, day,...",Severe weather disruptions
7,05/2018,BORDEAUX (AFP),'Unprecedented' hailstorm in France hits Borde...,https://www.straitstimes.com/world/europe/unpr...,BORDEAUX (AFP) - Winemakers in western France ...,"area, association, areas, affected, told, wine...",Severe weather disruptions
8,03/2018,IPOH (THE STAR/ASIA NEWS NETWORK),Hailstorm in Ipoh shocks residents,https://www.straitstimes.com/asia/se-asia/hail...,IPOH (THE STAR/ASIA NEWS NETWORK) - A freak th...,"residents, car, shocks, ipoh, son, hailstorm, ...",Severe weather disruptions
9,08/2017,A Ukrainian pilot has been hailed a hero and a...,Hero pilot lands passenger plane 'blind' in Tu...,https://www.straitstimes.com/world/europe/hero...,A Ukrainian pilot has been hailed a hero and a...,"plane, turkey, viral, blind, safely, passenger...",Severe weather disruptions
